In [1]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_double_side_view, show_selected_joints)
import vis.fields
import matplotlib.cm as cm
import time

In [2]:
rod_length = 0.09982061776030171
width = rod_length / 15 * 5
thickness = width / 5 * 0.35
print(width, thickness)
io = InputOrganizer('new_kitty_1', thickness, width, weaving_dir)
#  [ ear 214, ear 255, nose 39]

0.033273539253433904 0.0023291477477403733


In [3]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 0.01
sw = 0.01

In [4]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = get_double_side_view(curved_linkage, flip = True)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…

In [5]:
curved_linkage.setExtendedDoFsPSRL(np.load('new_kitty_1_dof.npy'))

In [6]:
bottomColor =[180/255., 158/255., 119/255.]
topColor =[79/255., 158/255., 246/255.]
heights = curved_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [7]:
def eqm_callback(prob, i):
    curved_linkage_view.update(scalarField = colors)

In [8]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)

0	0.00148561	1.44543	1.44543	1	1
1	0.000224213	0.105055	0.105055	1	1
2	0.000157125	0.0285207	0.0285207	1	1
3	0.000123938	0.0133325	0.0133325	1	1
4	0.000100982	0.00656026	0.00656026	1	1
5	8.48006e-05	0.00331926	0.00331926	1	1
6	7.41592e-05	0.00187758	0.00187758	0.5	0
7	5.79477e-05	0.00711136	0.00711136	1	0
8	5.08861e-05	0.00538869	0.00538869	1	0
9	5.06684e-05	0.00693253	0.00693253	1	1
10	5.04221e-05	0.000208268	0.000208268	1	0
11	5.02992e-05	0.00257228	0.00257228	1	0
12	5.02807e-05	0.000565988	0.000565988	1	0
13	5.02776e-05	7.30268e-05	7.30268e-05	1	0
14	5.02775e-05	1.32571e-05	1.32571e-05	1	0
15	5.02775e-05	8.48089e-09	8.48089e-09	1	0


In [9]:
def visualize_crossing(index, linkage, view):
    vf = show_selected_joints(linkage, [index], flip = True)
    view.update(vectorField = vf, scalarField = colors)

In [10]:
ribbons = order_segments_by_ribbons(curved_linkage)
_, _, _, _, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, curved_linkage, rest = True)
all_joint_index_list = [j_list + [j_list[0]] for j_list in all_joint_index]

def get_ribbon_crossing_list(index, linkage, view):
    selected_list = []
    selected_ribbon = []
    for ribbon_index, index_list in enumerate(all_joint_index_list):
        if index in set(index_list):
            selected_ribbon.append(ribbon_index)
            selected_list.append(index_list)
    print("The crossing {} belongs to ribbon {}".format(index, ', '.join([str(x) for x in selected_ribbon])))
    for i in range(len(selected_list)):
        print('Ribbon {}: {}'.format(selected_ribbon[i], selected_list[i]))

In [15]:
def select_crossing(crossing_index):
    visualize_crossing(crossing_index, curved_linkage, curved_linkage_view)
    get_ribbon_crossing_list(crossing_index, curved_linkage, curved_linkage_view)
select_crossing(85)

The crossing 85 belongs to ribbon 2
Ribbon 2: [40, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 25, 93, 94, 95, 96, 97, 98, 62, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 5, 117, 118, 119, 120, 121, 46, 122, 123, 124, 125, 126, 127, 128, 129, 130, 88, 131, 132, 133, 134, 30, 135, 136, 137, 138, 139, 140, 141, 142, 143, 72, 144, 145, 146, 111, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 15, 161, 162, 53, 163, 164, 165, 166, 167, 168, 128, 169, 170, 85, 171, 172, 173, 174, 175, 36, 176, 177, 178, 179, 73, 144, 180, 181, 108, 182, 183, 184, 185, 186, 187, 188, 189, 60, 190, 191, 192, 193, 23, 194, 195, 196, 197, 198, 199, 129, 169, 200, 201, 202, 203, 204, 205, 206, 42, 207, 208, 4, 116, 209, 210, 211, 212, 213, 214, 153, 215, 216, 217, 218, 219, 220, 221, 222, 223, 189, 61, 98, 224, 225, 226, 227, 228, 229, 230, 231, 29, 134, 232, 233, 234, 235, 86, 170, 200, 236, 237, 238, 124, 239, 240, 49, 241, 242,

In [18]:
def write_crossing_ribbon_info(linkage, filename, scale):
    ''' return the list of two ribbon indices per crossing and the list of crossing lists per ribbon
    '''
    ribbons = order_segments_by_ribbons(linkage)
    _, _, _, _, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, linkage, rest = True)
    all_joint_index_list = [j_list + [j_list[0]] for j_list in all_joint_index]

    def get_ribbon_crossing_list(index):
        selected_list = []
        selected_ribbon = []
        for ribbon_index, index_list in enumerate(all_joint_index_list):
            if index in set(index_list):
                selected_ribbon.append(ribbon_index)
                selected_list.append(index_list)
        # print("The crossing {} belongs to ribbon {}".format(index, ', '.join([str(x) for x in selected_ribbon])))
        # for i in range(len(selected_list)):
        #     print('Ribbon {}: {}'.format(selected_ribbon[i], selected_list[i]))
        return selected_ribbon
    pairs_of_ribbons_per_crossing = [get_ribbon_crossing_list(i) for i in range(linkage.numJoints())]

    with open(filename, 'w') as f:
        f.write('# crossing count {}, ribbon count {}'.format(linkage.numJoints(), len(ribbons)))
        for vx in linkage.jointPositions().reshape((linkage.numJoints(), 3)):
            vx *= scale
            f.write('v  {} {} {}\n'.format(vx[0], vx[1], vx[2]))
        for c_index, pair in enumerate(pairs_of_ribbons_per_crossing):
            if len(pair) == 1:
                f.write('crossing {}: {}\n'.format(c_index, pair[0]))
            else:
                if len(pair) != 2:
                    print("Something is wrong")
                f.write('crossing {}: {} {}\n'.format(c_index, pair[0], pair[1]))
        for r_index, index_list in enumerate(all_joint_index_list):
            f.write('ribbon {}: {}\n'.format(r_index, " ".join([str(x) for x in index_list])))

    return pairs_of_ribbons_per_crossing, all_joint_index_list

In [19]:
two_ribbon_per_crossing, crossing_list_per_ribbon = write_crossing_ribbon_info(curved_linkage, 'cat_webapp_output.txt', 100)